In [1]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')

HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [2]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


In [3]:
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


In [4]:
from indox.data_connector import GutenbergReader

reader = GutenbergReader()

book_id = "11"  # Alice's Adventures in Wonderland
book = reader.get_book(book_id)

In [6]:
content = book.content

In [7]:
from indox.splitter import semantic_text_splitter
content_chunks = semantic_text_splitter(content,500)

In [8]:
from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed)
indox.connect_to_vectorstore(vectorstore_database=db)

INFO: Connection to the vector store database established successfully


In [9]:
indox.store_in_vectorstore(docs= content_chunks)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


In [11]:
query = "Who is the speaker talking to in the text?"
retriever = indox.QuestionAnswer(vector_database=db, llm=mistral_qa, top_k=5)

In [12]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [13]:
answer

"The speaker is talking to themselves in the text. The passage is Alice's internal monologue."